In [ ]:
from agent import QLearningAgent
from verify_data import FishGame, ParseError

In [ ]:
import os
import pickle

if os.path.isfile('memories.pkl'):
    with open('memories.pkl', 'rb') as f:
        memories = pickle.load(f)
else:
    memories = []
    for filename in os.listdir('data'):
        filepath = os.path.join('data', filename)
        with open(filepath, 'r') as f:
            try:
                print(f"{filename}")
                game = FishGame(f.readlines())
                for player in game.players:
                    for _ in range(10):
                        game.shuffle()
                        memories.append(game.memory(player))
            except ParseError as e:
                print(f"{filename}: {e}")
                break
    with open('memories.pkl', 'wb') as f:
        pickle.dump(memories, f)

In [ ]:
agent = QLearningAgent()
agent.train_on_data(memories, 0, 50)

In [ ]:
batch = agent.unpack_memory([memories[3][15]])
agent.hand_predictor(agent.tensor(batch['state']), agent.action_masks(*batch['mask_dep'].values())['hands'])

In [ ]:
from datetime import datetime

os.makedirs('models', exist_ok=True)
model_path = f'models/fish_agent_{datetime.now().strftime("%Y%m%d_%H%M%S")}.pth'
agent.save_model(model_path)

In [ ]:
agent = QLearningAgent() 
agent.load_model('models/fish_agent_20250417_192317.pth')

In [ ]:
agent.train_self_play(2000, update_rate=1, hand_epochs=5, q_epochs=5, path='models/fish_agent_20250417_192317.pth')